In [1]:
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from sklearn.preprocessing import StandardScaler

c:\Users\eria\Anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
from tps_feb_2021.config import config
from tps_feb_2021.utils import add_noise, save_model, get_run_logdir, extract_features

In [3]:
data_dir = config['data_dir']

train_raw = pd.read_csv(data_dir + 'raw/train.csv')
test_raw = pd.read_csv(data_dir + 'raw/test.csv')


In [4]:
cont_cols = [col for col in train_raw.columns if col[:4] == 'cont']
cat_cols = [col for col in train_raw.columns if col[:3] == 'cat']

In [5]:
X = pd.concat([train_raw[cat_cols + cont_cols], test_raw[cat_cols + cont_cols]])
X.head()

,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,...,cont4,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13
0,A,B,A,A,B,D,A,E,C,I,...,0.281421,0.881122,0.421650,0.741413,0.895799,0.802461,0.724417,0.701915,0.877618,0.719903
1,B,A,A,A,B,B,A,E,A,F,...,0.282354,0.440011,0.346230,0.278495,0.593413,0.546056,0.613252,0.741289,0.326679,0.808464
2,A,A,A,C,B,D,A,B,C,N,...,0.293756,0.914155,0.369602,0.832564,0.865620,0.825251,0.264104,0.695561,0.869133,0.828352
3,A,A,A,C,B,D,A,E,G,K,...,0.769785,0.934138,0.578930,0.407313,0.868099,0.794402,0.494269,0.698125,0.809799,0.614766
4,A,B,A,A,B,B,A,E,C,F,...,0.279105,0.382600,0.705940,0.325193,0.440967,0.462146,0.724447,0.683073,0.343457,0.297743


In [6]:
X.shape

(500000, 24)

In [7]:
noisy_X = add_noise(X, p=0.15)

In [8]:
scaler = StandardScaler()
cont_X_scaled = pd.DataFrame(scaler.fit_transform(X[cont_cols]), columns=cont_cols)
cont_noisy_X_scaled = pd.DataFrame(scaler.fit_transform(noisy_X[cont_cols]), columns=cont_cols)

In [9]:
noisy_one_hot_cats = pd.get_dummies(noisy_X[cat_cols])
one_hot_cats = pd.get_dummies(X[cat_cols])

In [10]:
noisy_X = pd.concat([noisy_one_hot_cats, cont_noisy_X_scaled], axis=1)
X = pd.concat([one_hot_cats.reset_index(drop=True), cont_X_scaled.reset_index(drop=True)], axis=1)

In [11]:
noisy_X.shape

(500000, 70)

In [12]:
X.shape

(500000, 70)

In [13]:
noisy_X.columns

Index(['cat0_A', 'cat0_B', 'cat1_A', 'cat1_B', 'cat2_A', 'cat2_B', 'cat3_A',
       'cat3_B', 'cat3_C', 'cat3_D', 'cat4_A', 'cat4_B', 'cat4_C', 'cat4_D',
       'cat5_A', 'cat5_B', 'cat5_C', 'cat5_D', 'cat6_A', 'cat6_B', 'cat6_C',
       'cat6_D', 'cat6_E', 'cat6_G', 'cat6_H', 'cat6_I', 'cat7_A', 'cat7_B',
       'cat7_C', 'cat7_D', 'cat7_E', 'cat7_F', 'cat7_G', 'cat7_I', 'cat8_A',
       'cat8_B', 'cat8_C', 'cat8_D', 'cat8_E', 'cat8_F', 'cat8_G', 'cat9_A',
       'cat9_B', 'cat9_C', 'cat9_D', 'cat9_E', 'cat9_F', 'cat9_G', 'cat9_H',
       'cat9_I', 'cat9_J', 'cat9_K', 'cat9_L', 'cat9_M', 'cat9_N', 'cat9_O',
       'cont0', 'cont1', 'cont2', 'cont3', 'cont4', 'cont5', 'cont6', 'cont7',
       'cont8', 'cont9', 'cont10', 'cont11', 'cont12', 'cont13'],
      dtype='object')

In [14]:
X.columns

Index(['cat0_A', 'cat0_B', 'cat1_A', 'cat1_B', 'cat2_A', 'cat2_B', 'cat3_A',
       'cat3_B', 'cat3_C', 'cat3_D', 'cat4_A', 'cat4_B', 'cat4_C', 'cat4_D',
       'cat5_A', 'cat5_B', 'cat5_C', 'cat5_D', 'cat6_A', 'cat6_B', 'cat6_C',
       'cat6_D', 'cat6_E', 'cat6_G', 'cat6_H', 'cat6_I', 'cat7_A', 'cat7_B',
       'cat7_C', 'cat7_D', 'cat7_E', 'cat7_F', 'cat7_G', 'cat7_I', 'cat8_A',
       'cat8_B', 'cat8_C', 'cat8_D', 'cat8_E', 'cat8_F', 'cat8_G', 'cat9_A',
       'cat9_B', 'cat9_C', 'cat9_D', 'cat9_E', 'cat9_F', 'cat9_G', 'cat9_H',
       'cat9_I', 'cat9_J', 'cat9_K', 'cat9_L', 'cat9_M', 'cat9_N', 'cat9_O',
       'cont0', 'cont1', 'cont2', 'cont3', 'cont4', 'cont5', 'cont6', 'cont7',
       'cont8', 'cont9', 'cont10', 'cont11', 'cont12', 'cont13'],
      dtype='object')

In [29]:
model = keras.models.Sequential([
    keras.layers.Input(shape=noisy_X.shape[1:]),
    keras.layers.Dense(500, activation='relu'),
    keras.layers.Dense(500, activation='relu'),
    keras.layers.Dense(500, activation='relu'),
    keras.layers.Dense(70)
])

model.compile(loss="mean_squared_error", optimizer=keras.optimizers.Adam(learning_rate=0.001))

In [30]:
run_logdir = get_run_logdir()
tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)

In [31]:
early_stopping_cb = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=10)

In [32]:
history = model.fit(noisy_X, X, epochs=2001,
                    callbacks=[tensorboard_cb, early_stopping_cb])

Epoch 1/2001
15625/15625 [==============================] - 75s 5ms/step - loss: 0.0644
Epoch 2/2001
15625/15625 [==============================] - 72s 5ms/step - loss: 0.0604
Epoch 3/2001
15625/15625 [==============================] - 69s 4ms/step - loss: 0.0594
Epoch 4/2001
15625/15625 [==============================] - 71s 5ms/step - loss: 0.0589
Epoch 5/2001
15625/15625 [==============================] - 70s 5ms/step - loss: 0.0587
Epoch 6/2001
15625/15625 [==============================] - 70s 5ms/step - loss: 0.0585
Epoch 7/2001
15625/15625 [==============================] - 70s 5ms/step - loss: 0.0584
Epoch 8/2001
15625/15625 [==============================] - 71s 5ms/step - loss: 0.0583
Epoch 9/2001
15625/15625 [==============================] - 71s 5ms/step - loss: 0.0582
Epoch 10/2001
15625/15625 [==============================] - 73s 5ms/step - loss: 0.0581
Epoch 11/2001
15625/15625 [==============================] - 77s 5ms/step - loss: 0.0581
Epoch 12/2001
15625/15625 [===

In [33]:
save_model(model, '05_dae_model_cont_scaled.h5')

In [15]:
model = model = keras.models.load_model('../models/05_dae_model_cont_scaled.h5')

In [16]:
features_df = extract_features(model, noisy_X)

15625/15625 [==============================] - 50s 3ms/step


In [17]:
features_df

,0,1,2,3,4,5,6,7,8,9,...,1490,1491,1492,1493,1494,1495,1496,1497,1498,1499
0,1.254820,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
1,0.599574,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
2,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
3,1.573088,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.759173,0.0
4,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499995,1.272923,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.790421,0.0
499996,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
499997,0.125109,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.722705,0.0
499998,1.289798,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
